In [ ]:
#--- IMPORT DEPENDENCIES ------------------------------------------------------+

from __future__ import division
import random
import math

#--- COST FUNCTION ------------------------------------------------------------+

# function we are attempting to optimize (minimize)
def func1(x):
    total=0
    for i in range(len(x)):
        total+=x[i]**2
    return total

#--- MAIN ---------------------------------------------------------------------+

class Particle:
    def __init__(self,x0):
        self.position_i=[]          # particle position
        self.velocity_i=[]          # particle velocity
        self.pos_best_i=[]          # best position individual
        self.score_best_i=-1          # best score individual
        self.score_i=-1               # score individual

        for i in range(0,num_dimensions):
            self.velocity_i.append(random.uniform(-1,1))
            self.position_i.append(x0[i])

    # evaluate current fitness
    def evaluate(self,costFunc):
        self.score_i, self.q, self.PLR, self.EE=costFunc(self.position_i)

        # check to see if the current position is an individual best
        if self.score_i < self.score_best_i or self.score_best_i==-1:
            self.pos_best_i=self.position_i
            self.score_best_i=self.score_i

    # update new particle velocity
    def update_velocity(self,pos_best_g):
        w=0.5       # constant inertia weight (how much to weigh the previous velocity)
        c1=1        # cognative constant
        c2=2        # social constant

        for i in range(0,num_dimensions):
            r1=random.random()
            r2=random.random()

            vel_cognitive=c1*r1*(self.pos_best_i[i]-self.position_i[i])
            vel_social=c2*r2*(pos_best_g[i]-self.position_i[i])
            self.velocity_i[i]=w*self.velocity_i[i]+vel_cognitive+vel_social

    # update the particle position based off new velocity updates
    def update_position(self,bounds):
        for i in range(0,num_dimensions):
            self.position_i[i]=self.position_i[i]+self.velocity_i[i]

            # adjust maximum position if necessary
            if self.position_i[i]>bounds[i][1]:
                self.position_i[i]=bounds[i][1]

            # adjust minimum position if neseccary
            if self.position_i[i] < bounds[i][0]:
                self.position_i[i]=bounds[i][0]
        total = sum(self.position_i)
        for i in range(len(self.position_i)):
          self.position_i[i]=self.position_i[i]/total

class PSO():
    def __init__(self,costFunc,x0,bounds,num_particles,maxiter):
        global num_dimensions

        num_dimensions=len(x0)
        score_best_g=-1                   # best error for group
        pos_best_g=[]                   # best position for group
        self.best_scores_g = []
        self.PLR = 0
        self.EE = 0
        q=-100
        # establish the swarm
        swarm=[]
        # for i in range(0,num_particles):
        #   swarm.append(Particle(x0))
        swarm.append(Particle(x0))
        n = len(x0)
        for i in range(0,num_particles-1):
          random_particle =[]
          for i in range(n):
            random_particle.append(random.random())
          total = sum(random_particle)
          for i in range(n):
            random_particle[i] = random_particle[i]/total
          swarm.append(Particle(random_particle))

        # begin optimization loop
        i=0
        while i < maxiter:
            #print i,err_best_g
            # cycle through particles in swarm and evaluate fitness
            for j in range(0,num_particles):
                swarm[j].evaluate(costFunc)
                # determine if current particle is the best (globally)
                if swarm[j].score_i > score_best_g or score_best_g == -1:
                    pos_best_g=list(swarm[j].position_i)
                    score_best_g=float(swarm[j].score_i)
                    q=swarm[j].q
                    PLR = swarm[j].PLR
                    EE = swarm[j].EE
                # print(i, j, swarm[j].position_i, pos_best_g, score_best_g)
            # print(i, pos_best_g, score_best_g, q, PLR)
            # cycle through swarm and update velocities and position
            for j in range(0,num_particles):
                swarm[j].update_velocity(pos_best_g)
                swarm[j].update_position(bounds)
            i+=1
            self.best_scores_g.append(score_best_g)

        # print final results
        # print('FINAL:',pos_best_g, score_best_g, q, PLR)
        self.PLR = PLR
        self.EE = EE


In [ ]:
import numpy as np
M=32 # Users
N=15# timeslots
# G = M/N

Gs = np.arange(0.6, 3, 0.1)
Ls = list(range(1,4))
Save_PLR = []
Save_throughput = []
Save_EE = []
for i in range(len(Ls)):
  L = Ls[i]
  # print('Power Level :', L)
  Save_PLR.append([])
  Save_throughput.append([])
  Save_EE.append([])
  for G in Gs:
    # print('G value :', G)
    eq = [0, 0.5112, 0.266, 0, 0, 0, 0, 0.2228] # 0.5112x^2 + 0.266x^3 + 0.2228x^8
    class Lambda():
      def __init__(self, LAMBDA = eq):
        total = sum(eq)
        self.LAMBDA=[]
        for i in range(len(LAMBDA)):
          self.LAMBDA.append(LAMBDA[i]/total)
        self.Lambda = self.deviation()
      def deviation(self):
        temp = 0
        for i in range(len(self.LAMBDA)):
          temp = temp + self.LAMBDA[i] * (i+1)
        return temp

    class cal_pi():
      def __init__(self, G, Lambda, L, LAMBDA = eq):
        q0 = 1
        self.G = G
        self.L = L
        self.p0 = np.exp(-q0*G*Lambda)*self.factorial(L-1)
        self.Lambda = Lambda
        self.LAMBDA = LAMBDA
        summation = 0
        for r in range(1, L+1):
          summation = summation + (q0*G*Lambda/L)**(r-1)/(self.factorial(r-1)*self.factorial(L-r))
        self.p0 = 1 - self.p0 * summation
        self.pi = self.p0
      def factorial(self, n):
        temp = 1
        for i in range(n):
          temp = temp*(i+1)
        return temp

      def cal_LAMBDA(self, x):
        temp = 0
        for i in range(len(eq)):
          temp = temp + self.LAMBDA[i] * x ** (i+1)
        return temp

      def cal_pi(self):
        LAMBDA = self.cal_LAMBDA(self.pi)
        self.pi = np.exp(-LAMBDA*self.G*self.Lambda)*self.factorial(self.L-1)
        summation = 0
        for r in range(1, self.L+1):
          summation = summation + (LAMBDA*self.G*self.Lambda/self.L)**(r-1)/(self.factorial(r-1)*self.factorial(self.L-r))
        self.pi = 1 - self.pi * summation
        return self.pi

    target_err = 1e-7
    def func2(x):
      lam = Lambda(LAMBDA=x)
      Lamda = lam.Lambda
      pi= cal_pi(G, Lamda, L=L, LAMBDA = x)
      q = 1
      n=len(x)
      for i in range(30): #IRSA iteration
        temp = pi.cal_pi()
        PLR = 0
        temp_EE = 0
        for j in range(n):
          PLR +=x[j]*q**(j+1)
          if L==1:
            temp_EE += x[j]*(j+1)
          elif L==2:
            temp_EE += 0.5*0.5*x[j]*(j+1) + 0.5*1*x[j]*(j+1)
          elif L==3:
            temp_EE += 1/3*0.25*x[j]*(j+1) + 1/3*0.5*x[j]*(j+1) + 1/3*1*x[j]*(j+1)
        if q <= temp and q !=0 and temp !=0:
          EE = (1-PLR)/temp_EE
          return -10, q, PLR, EE
        elif temp !=0:
          q=temp
          # EE = (1-PLR)/temp_EE
      EE = (1-PLR)/temp_EE
      # print('EE', EE, 1-PLR)
      del lam
      del pi
      if PLR> target_err:
        return -10, q, PLR, EE
      else:
        temp =0
        for i in range(n):
          temp = temp + x[i]*(i+1)
        return (1-q)/temp, q, PLR, EE

    bounds=[(0,1),(0,1),(0,1),(0,1),(0,1),(0,1),(0,1),(0,1),(0,1)]  # input bounds [(x1_min,x1_max),(x2_min,x2_max)...]
    test_result = PSO(func2,eq,bounds,num_particles=30,maxiter=30)
    Save_PLR[i].append(test_result.PLR)
    Save_throughput[i].append(G*(1-test_result.PLR))
    Save_EE[i].append(test_result.EE)
    del test_result
#     print()
# print(Save_results)

In [ ]:
#PLR
for i in range(len(Ls)):
  L = Ls[i]
  print('Power Level :', L)
  print(Save_PLR[i])
  print()

#Throughput
for i in range(len(Ls)):
  L = Ls[i]
  print('Power Level :', L)
  print(Save_throughput[i])
  print()

#Energy Efficiency
#Throughput
for i in range(len(Ls)):
  L = Ls[i]
  print('Power Level :', L)
  print(Save_EE[i])
  print()

#L=1일때 0.6~0.9까지
#L=2일때 1.4~1.7까지
#L=3일때 1.8~2.1까지 해서 Fitness Scores 그래프 그리자

#Energy Efficiency도 위 조건에서

#PLR도 위 조건에서

#Throughput도 위 조건에서
# Throughput은 G(1-PLR)임.

Power Level : 1
[9.999503876815402e-08, 9.999721696659318e-08, 9.999511462725375e-08, 9.999655599102947e-08, 9.999902046899077e-08, 0.9061654030635031, 0.9403533576816883, 0.9609124119669704, 0.9739023838671241, 0.9823581334690039, 0.9879712764495161, 0.9917481887242415, 0.994314119620991, 0.9960695145912924, 0.997276533691131, 0.9981095987757049, 0.9986861576215822, 0.9990860060269775, 0.9993637233116281, 0.9995568289632912, 0.9996912120460107, 0.9997847865709366, 0.9998499740805868, 0.9998954010076941]

Power Level : 2
[2.220446049250313e-16, 1.965094753586527e-14, 1.0529355165544985e-12, 3.514633029055858e-11, 8.045699662062589e-10, 1.3550103705739502e-08, 9.999844533080579e-08, 9.999319305496848e-08, 9.999835744347238e-08, 9.99988074262013e-08, 0.9437712153934921, 0.9615269747176454, 0.9732455623734326, 0.9811989176011598, 0.9866947873186741, 0.9905388869218757, 0.993250338996989, 0.9951741426729956, 0.9965447110489629, 0.9975239028265187, 0.9982248049146923, 0.9987271146908852, 0.

In [ ]:
# Fixed랑 EE 비교 그래프 만들자